In [1]:
#_LCS
import numpy as np
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

class LCS:
    
    def __init__(self):
        
        pass

In [2]:
    def _FTLE_(self):
        
        if hasattr(self, 'C') == False:
        
            self._cauchy_green_strain()
            
        print("=================FTLE=================")
    
        self.FTLE = np.zeros((self.len_Y, self.len_X, self.dim))*np.nan
        
        for i in range(self.len_Y):
        
            for j in range(self.len_X):
                
                lambda_min, lambda_max, v_min, v_max = self._eigenvalues_and_eigenvectors(self.C[i, j, :, :])
                        
                if lambda_min == 0:
                        
                    self.FTLE[i, j, :] = np.nan
                            
                else:
                
                    self.FTLE[i, j, 0] = 1/(2*(self.lenT))*np.log(lambda_min)
                    self.FTLE[i, j, 1] = 1/(2*(self.lenT))*np.log(lambda_max)
                        
        return self.FTLE[:,:,0], self.FTLE[:,:,1]

In [3]:
    def _PRA_(self):
            
        if hasattr(self, 'grad_Fmap_grid') == False:
        
            self._grad_Fmap_grid()
            
        print("=================PRA=================")
            
        self.PRA = np.zeros(self.X_domain.shape)
            
        for i in range(self.len_Y):
        
            for j in range(self.len_X):
                
                U, S, V = self._svd(self.grad_Fmap_grid[i, j, :, :])
                
                self.PRA[i, j] = np.arccos(U[0, 0]*V[0, 0]+U[0, 1]*V[0, 1])
        
        return self.PRA

In [4]:
    def _LAVD_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()
            
        print("=================LAVD=================")
            
        def parallelization(k, t):
                
            self.omega = self._vorticity(t)
                
            spatially_averaged_vorticity = np.nanmean(self.omega.ravel())
                
            LVD = np.zeros((self.len_Y, self.len_X))
            
            for i in range(self.len_Y):
            
                for j in range(self.len_X):
                    
                    x = np.array([self.trajectory_grid[i, j, 0, k], self.trajectory_grid[i, j, 1, k]]).reshape(1, -1)

                    W = self._vorticity_tensor(x, t)
                    
                    omega = W[0, 1]-W[1, 0]
                
                    LVD[i, j] = np.abs(omega-spatially_averaged_vorticity)
                    
            return LVD
        
        self.LVD = np.array(Parallel(n_jobs=self.Ncores, verbose = 1)(delayed(parallelization)(k, t) for k, t in tqdm(enumerate(self.time), total=len(self.time))))
        
        self.LAVD = np.nanmean(self.LVD, axis = 0)
        
        return self.LAVD

In [5]:
    def _TRA_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()

        print("=================TRA=================")
        
        self.TRA = np.zeros((self.len_Y, self.len_X))
            
        for i in range(self.len_Y):
            
            for j in range(self.len_X):
                    
                velx0 = self.velocity_grid[i, j, 0, 0]
                vely0 = self.velocity_grid[i, j, 1, 0]
                    
                vel0 = np.sqrt(velx0**2+vely0**2)
                    
                velxN = self.velocity_grid[i, j, 0, -1]
                velyN = self.velocity_grid[i, j, 1, -1]
                
                vel1 = np.sqrt(velxN**2+velyN**2)
                
                self.TRA[i, j] = np.abs(np.arccos((velx0*velxN+vely0*velyN)/(vel0*vel1)))
        
        return self.TRA

In [6]:
    def _TRA_bar_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()

        print("=================TRA_bar=================")
        
        self.TR_bar = np.zeros((self.len_Y, self.len_X, self.trajectory_grid.shape[3]))
        
        for k in tqdm(range(self.lenT-2), total = self.lenT-2):
            
            for i in range(self.len_Y):
            
                for j in range(self.len_X):
                    
                    velx0 = self.velocity_grid[i, j, 0, k]
                    vely0 = self.velocity_grid[i, j, 1, k]
                    
                    vel0 = np.sqrt(velx0**2+vely0**2)
                    
                    velx1 = self.velocity_grid[i, j, 0, k + 1]
                    vely1 = self.velocity_grid[i, j, 1, k + 1]
                
                    vel1 = np.sqrt(velx1**2+vely1**2)
                
                    self.TR_bar[i, j, k-1] = np.abs(np.arccos((velx0*velx1+vely0*vely1)/(vel0*vel1)))
        
        self.TRA_bar = np.nanmean(self.TR_bar, axis = 2)/(self.lenT)
        
        return self.TRA_bar

In [7]:
    def _TSE_(self):
    
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()

        print("=================TSE=================")
        
        self.TSE = np.zeros((self.len_Y, self.len_X))
            
        for i in range(self.len_Y):
            
            for j in range(self.len_X):
                    
                velx0 = self.velocity_grid[i, j, 0, 0]
                vely0 = self.velocity_grid[i, j, 1, 0]
                    
                vel0 = np.sqrt(velx0**2+vely0**2)
                    
                velxN = self.velocity_grid[i, j, 0, -1]
                velyN = self.velocity_grid[i, j, 1, -1]
                
                velN = np.sqrt(velxN**2+velyN**2)
                
                self.TSE[i, j] = 1/(self.tN-self.t0)*np.log(velN/vel0)
        
        return self.TSE

In [8]:
    def _TSE_bar_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()

        print("=================TSE_bar=================")
        
        self.TSE_bar = np.zeros((self.len_Y, self.len_X, self.lenT-2))
        
        for k in tqdm(range(self.lenT-2), total = self.lenT-2):
            
            for i in range(self.len_Y):
            
                for j in range(self.len_X):
                    
                    velx0 = self.velocity_grid[i, j, 0, k]
                    vely0 = self.velocity_grid[i, j, 1, k]
                    
                    vel0 = np.sqrt(velx0**2+vely0**2)
                    
                    velx1 = self.velocity_grid[i, j, 0, k + 1]
                    vely1 = self.velocity_grid[i, j, 1, k + 1]
                
                    vel1 = np.sqrt(velx1**2+vely1**2)
                
                    self.TSE_bar[i, j, k] = np.abs(np.log(vel1/vel0))
        
        self.TSE_bar = np.nanmean(self.TSE_bar, axis = 2)/(self.lenT)
        
        return self.TRA_bar

In [9]:
    def _hyperbolic_LCS_local_variational_theory(self):
        
        print("=================Hyperbolic LCS from local variational theory=================")

In [10]:
    def _elliptic_LCS_local_variational_theory(self):
        
        print("=================Elliptic LCS from local variational theory=================")

In [11]:
    def _hyperbolic_LCs_global_variational_theory(self):
        
        print("=================Hyperbolic LCS from global variational theory=================")

In [12]:
    def _elliptic_LCs_global_variational_theory(self):
        
        print("=================Elliptic LCS from global variational theory=================")

In [13]:
    def _vorticity(self, t):
        
        self.omega = np.zeros((self.len_Y, self.len_X))
        
        for i in range(self.len_Y):
            
            for j in range(self.len_X):
                
                x = np.array([self.X_domain[i, j], self.Y_domain[i, j]]).reshape(1, -1)
    
                W = self._vorticity_tensor(x, t)
                
                self.omega[i, j] = W[0, 1]-W[1, 0]
                
        return self.omega              

In [14]:
    def _ridge_trench(self, Field, threshold = None, type = "ridge", method = "threshold", resolution = 1, ds = 1, n_iterations = 100):
        
        if type == "ridge":
            
            sign = 1
            
            if threshold is None:
            
                print("Threshold value is None --> Specify threshold.")
                print("If not specified, threshold is set to ", 0.1, " of the maximum value of the given scalar field")
            
                threshold = .1*np.nanmax(Field)
            
        elif type == "trench":
            
            sign = -1
                  
            if threshold is None:
            
                print("Threshold value is None --> Specify threshold.")
                print("If not specified, threshold is set to ", 0.1, " of the maximum value of the given scalar field")
            
                threshold = .1*np.nanmin(Field)
            
        if method == "threshold":
            
            mask = (Field >= threshold)
            
            extrema_x = self.X_domain[mask].ravel()
            extrema_y = self.Y_domain[mask].ravel()
            
            return extrema_x, extrema_y
            
        elif method == "gradient":
            
            Field[np.isnan(Field)] = 0
            
            Interpolant_Field = self._Interpolation(self.Y_domain, self.X_domain, Field, method = "cubic")
            
            grad_Field = np.zeros((Field.shape[0], Field.shape[1], 2))
            
            for i in range(1, Field.shape[0]-1):
                
                for j in range(1, Field.shape[1]-1):
                    
                    dy = (self.Y_domain[i+1,0] - self.Y_domain[i-1, 0])/2
                    dx = (self.X_domain[0, j+1] - self.X_domain[0, j-1])/2

                    grad_Field[i, j, 0] = (Interpolant_Field(self.Y_domain[i, j], self.X_domain[i, j]+.1*dx)[0][0]-Interpolant_Field(self.Y_domain[i, j], self.X_domain[i, j]-.1*dx)[0][0])/(2*.1*dx)
                    grad_Field[i, j, 1] = (Interpolant_Field(self.Y_domain[i, j]+.1*dy, self.X_domain[i, j])[0][0]-Interpolant_Field(self.Y_domain[i, j]-.1*dy, self.X_domain[i, j])[0][0])/(2*.1*dy)
        
            grad_Fieldx = grad_Field[:, :, 0]
            grad_Fieldy = grad_Field[:, :, 1]
            grad_Fieldx[np.isnan(grad_Fieldx)] = 0
            grad_Fieldy[np.isnan(grad_Fieldy)] = 0
            
            Interpolant_gradx_Field = self._Interpolation(self.Y_domain, self.X_domain, grad_Fieldx)
            Interpolant_grady_Field = self._Interpolation(self.Y_domain, self.X_domain, grad_Fieldy)
            
            x_grid = np.linspace(np.min(self.X_domain), np.max(self.X_domain), self.Y_domain.shape[0]*resolution)
            y_grid = np.linspace(np.min(self.Y_domain), np.max(self.Y_domain), self.Y_domain.shape[1]*resolution)
            
            extrema_x = []
            extrema_y = []
            
            for x in tqdm(x_grid, total = len(x_grid)):
                    
                for y in y_grid:
                    
                    x_eval = x
                    y_eval = y
    
                    loc = self._check_location(np.array([x_eval, y_eval]).reshape(1, -1))[0]
                        
                    gradient = 10
                    
                    iter = 0
                        
                    while iter < n_iterations and loc == "IN" and Interpolant_Field(y_eval, x_eval)[0][0] > threshold:
                            
                        loc = self._check_location(np.array([x_eval, y_eval]).reshape(1, -1))[0]
                        
                        gradx = Interpolant_gradx_Field(y_eval, x_eval)[0][0]
                        grady = Interpolant_grady_Field(y_eval, x_eval)[0][0]
                        
                        gradient = np.sqrt(gradx**2+grady**2)
                        
                        x_eval = x_eval + sign * ds * gradx/gradient*dx
                        y_eval = y_eval + sign * ds * grady/gradient*dy
                        
                        iter += 1
                    
                        extrema_x.append(x_eval)
                        extrema_y.append(y_eval)
            
            return extrema_x, extrema_y

        else:
            
            print("The method argument is not valid. Use either threshold or gradient")